In [1]:
import pandas as pd
import os
from glob import glob
import arcpy
from arcpy import env

In [2]:
wdir = ''
# import vulnerability matrix
vwmatrix = pd.read_csv('%s/vulnerability.csv' % wdir)
vwmatrix.set_index('stressors', inplace=True)

In [3]:
for yr in range(2003, 2014):
    for h in ['surface','mesopelagic','bathypelagic','abyssopelagic']:
        # list habitats files
        os.chdir("%s/Habitat/%s" % (wdir, h))
        habitats = sorted(glob('*.tif'))

        # list stressors files
        os.chdir("%s/Stressor/%s" % (wdir, yr))
        stressors = sorted(glob('*.tif'))    

        for sts in stressors:
            sts_name = sts[0 : len(sts) - 18]
            sts_path = "%s/Stressor/%s/%s" % (wdir, yr, sts)
            num_of_factors = len(habitats)

            # create expression
            expression = ''
            rasters = []
            input_names = []

            for hbt in habitats:
                hbt_name = hbt[len(h) + 1 : len(hbt) - 4]
                factor = str(vwmatrix.loc[sts_name, hbt_name])
                expression = expression + hbt_name + '*' + factor + '+'
                rasters.append(hbt)
                input_names.append(hbt_name)

            expression = expression[0 : len(expression) - 1]
            rasters.append(sts_path)
            input_names.append(sts_name)
            expression = ' ( %s ) * %s / %s' % (expression, sts_name, num_of_factors)

            # raster calculator
            env.workspace = "%s/Habitat/%s" % (wdir, h)
            output_raster = arcpy.ia.RasterCalculator(rasters, input_names, expression); 
            output_raster.save("%s/HI_%s" % (wdir, sts))
            print("%s: %s --- complete" % (h, sts_name))
        